In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F    
from torch.optim import AdamW
from torch.optim import ASGD
from torch.amp import GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import time
from functools import partial

import json
import os

CHECKPOINT_DIR = os.path.join("..", "checkpoints", "neural_bigram")
LOG_DIR = os.path.join("..", "logs", "neural_bigram")
RESULTS_DIR = os.path.join("..", "results", "neural_bigram")

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [3]:
# add parent directory to the path to allow imports
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from neural_bigram import NeuralBigram, ConfigNeuralBigram
from train_mj import train, evaluate, evaluate_ppl, ConfigTrain
from utils import init_dataloader, WarmupThenCosine, set_seed, count_params, save_checkpoint, load_checkpoint
from bpe_hf import train_bytelevel_bpe, train_and_encode_tokenizer, load_tokenizer, SPECIAL_TOKENS

## load data

In [4]:
data_dir = "../data/"
# without nl
# train_file = "Shakespeare_clean_train.txt"
# val_file = "Shakespeare_clean_valid.txt"
# test_file = "Shakespeare_clean_test.txt"

# with nl
train_file = "Shakespeare_clean_w_nl_train.txt"
val_file = "Shakespeare_clean_w_nl_valid.txt"
test_file = "Shakespeare_clean_w_nl_test.txt"

train_file = os.path.join(data_dir, train_file)
test_file = os.path.join(data_dir, test_file)
val_file = os.path.join(data_dir, val_file)

with open(train_file, "r", encoding="utf-8") as f:
    train_text = f.read().strip()

with open(val_file, "r", encoding="utf-8") as f:
    val_text = f.read().strip()

with open(test_file, "r", encoding="utf-8") as f:
    test_text = f.read().strip()

print("train text charachters length: ", len(train_text))
print("train text words count: ", len(train_text.split()))
print("train text first 100 chars: ", train_text[:100])

train text charachters length:  883780
train text words count:  158605
train text first 100 chars:  The Tragedy of Antony and Cleopatra


Dramatis Personae



MARK ANTONY
OCTAVIUS CAESAR
M. AEMILIUS L


## initialize tokensizer and data embeddings

In [5]:
N_MERGES = 200
MIN_FREQ = 2
SPECIAL_TOKENS = SPECIAL_TOKENS.copy()
PAD_TOKEN = SPECIAL_TOKENS.get("pad", "<pad>")
BOS_TOKEN = SPECIAL_TOKENS.get("bos", "<bos>")
EOS_TOKEN = SPECIAL_TOKENS.get("eos", "<eos>")


bpe_tokenizer = train_bytelevel_bpe(
    merges=N_MERGES,
    min_frequency=MIN_FREQ,
    files=[train_file],
    lowercase=False,
    add_prefix_space=True,
    special_tokens=SPECIAL_TOKENS,
    save_filename=f"shakespeare_bpe_{N_MERGES}"
)

encode = partial(bpe_tokenizer.encode, add_special_tokens=False)
decode = bpe_tokenizer.decode
vocab_size = bpe_tokenizer.get_vocab_size()
print("vocab size: ", vocab_size)
print("BPE tokenizer vocab: ", list(bpe_tokenizer.get_vocab().items())[:100])

PAD_TOKEN_ID = bpe_tokenizer.token_to_id(PAD_TOKEN) 
BOS_TOKEN_ID = bpe_tokenizer.token_to_id(BOS_TOKEN)
EOS_TOKEN_ID = bpe_tokenizer.token_to_id(EOS_TOKEN)

train_ids = encode(train_text).ids
val_ids = encode(val_text).ids
test_ids = encode(test_text).ids

print("train ids length: ", len(train_ids))
print("train ids first 100 ids: ", train_ids[:100])
print("train ids first 100 individual tokens: ", [bpe_tokenizer.id_to_token(i) for i in train_ids[:100]]) 

vocab size:  460
BPE tokenizer vocab:  [('°', 112), ('õ', 181), ('ł', 258), ('N', 49), ('ĥ', 229), ('â', 162), ('Ķ', 246), ('Þ', 158), ('ĠR', 391), ('<', 31), ('Æ', 134), ('Ĳ', 242), ("'s", 340), ('ú', 186), ('Ê', 138), ('Ġk', 381), ('Ġse', 401), ('3', 22), ('ge', 443), ('y', 92), ('Ã', 131), ('%', 8), ('¨', 105), ('T', 55), ('al', 366), ('ome', 363), ('C', 38), ('<bos>', 2), ('im', 338), ('Ġkn', 445), ('?', 34), ('d', 71), (':', 29), ('Ġthou', 369), ('id', 365), ('Ġyour', 373), ('/', 18), ('ad', 351), ('4', 23), ('es', 284), ('ing', 310), ('å', 165), ('ç', 167), ('¥', 102), ('st', 309), ('}', 96), ('1', 20), ('ĠO', 329), ('o', 82), ('c', 70), ('ö', 182), ('Ġne', 441), ('R', 53), ('Ö', 150), ('Đ', 208), ('Ġthis', 376), ('Ġon', 379), ('Ġwill', 400), ('Ģ', 226), ('ith', 336), ('ly', 394), ('.', 17), ('ĠT', 291), ('Ħ', 230), ('IA', 433), ('a', 68), ('ght', 354), ('Ġy', 292), ('ĠE', 345), ('AR', 396), ('k', 78), ('p', 83), ('D', 39), ('ĳ', 243), ('ha', 271), ('ď', 207), ('İ', 240), ('e', 7

## define the model's and training parameters (configs)

In [6]:
SEED = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# hyperparameters 
# tokenizer
# N_MERGES = 200
# MIN_FREQ = 2
# data
BATCH_SIZE = 32
BLOCK_SIZE = 128

#  model
DROPOUT = 0.1
# optimizer
LR = 3e-3
WEIGHT_DECAY = 1e-4
# lr scheduler
ETA_MIN = 1e-7

# training loop
EPOCHS = 60
ES_PATIENCE = 5
ES_TOLERANCE = 1e-6
GRAD_ACCUM_STEPS = 1
USE_AMP = False # IMPORTANT: set to False if training on CPU training 

# checkpoints
EVAL_INTERVAL = 2   # evaluate every n epochs
CKPT_INTERVAL = 10  # save checkpoint every n epochs

# make unique checkpoint file prefix from the parameters
ckpt_file_prefix = f"nbigram_bpe{N_MERGES}"
all_hparams_cfg_filename = f"{ckpt_file_prefix}_hparams.json"

# config for training
cfg_train = ConfigTrain(device=DEVICE,
                    epochs=EPOCHS, 
                    early_stop_patience=ES_PATIENCE,
                    early_stop_tolerance=ES_TOLERANCE,
                    grad_accum_steps=GRAD_ACCUM_STEPS,
                    # max_grad_norm=data.get("max_grad_norm", 1.0),
                    use_amp=USE_AMP,
                    seed=SEED,
                    eval_interval=EVAL_INTERVAL,
                    ckpt_interval=CKPT_INTERVAL,
                    ckpt_dir=CHECKPOINT_DIR,
                    ckpt_best_filename=f"{ckpt_file_prefix}_best.pt",
                    ckpt_last_filename=f"{ckpt_file_prefix}_last.pt",
                    log_dir=os.path.join(LOG_DIR, f"{ckpt_file_prefix}", f'{LR}_drop{DROPOUT}_{time.strftime("%y%m%d_%H%M%S")}'),
                    )
# config for model
cfg_model = ConfigNeuralBigram(
                    vocab_size=vocab_size,
                    dropout=DROPOUT
                    )

# model 
print("vocab size: ", vocab_size)
print("number of model parameters: ", count_params(NeuralBigram(config=cfg_model)))

# print all log and checkpoint dir and 
print("number of batches in train loader: ", len(train_ids)//BATCH_SIZE)
print("number of batches in test loader: ", len(test_ids)//BATCH_SIZE)
print("number of batches in val loader: ", len(val_ids)//BATCH_SIZE)

print("log dir: ", cfg_train.log_dir)
print("checkpoint dir: ", cfg_train.ckpt_dir)
print("checkpoint best filename: ", cfg_train.ckpt_best_filename)
print("checkpoint last filename: ", cfg_train.ckpt_last_filename)


vocab size:  460
number of model parameters:  211600
number of batches in train loader:  15570
number of batches in test loader:  1860
number of batches in val loader:  1849
log dir:  ../logs/neural_bigram/nbigram_bpe200/0.003_drop0.1_250824_215037
checkpoint dir:  ../checkpoints/neural_bigram
checkpoint best filename:  nbigram_bpe200_best.pt
checkpoint last filename:  nbigram_bpe200_last.pt


## Train the model

In [ ]:
set_seed(cfg_train.seed)
train_loader = init_dataloader(train_ids, BLOCK_SIZE, BATCH_SIZE, train=True, shuffle=True)
test_loader = init_dataloader(test_ids, BLOCK_SIZE, BATCH_SIZE, train=False, shuffle=True)
val_loader = init_dataloader(val_ids, BLOCK_SIZE, BATCH_SIZE, train=False, shuffle=True)


model = NeuralBigram(cfg_model)
model.to(cfg_train.device)

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)


lr_scheduler = WarmupThenCosine(optimizer, warmup_steps=500, T_max=cfg_train.epochs * len(train_loader) // max(1, cfg_train.grad_accum_steps), eta_min=ETA_MIN)
# rl_scheduler = CosineAnnealingLR(optimizer, T_max=cfg.epochs * len(train_loader) // max(1, cfg.grad_accum_steps), eta_min=ETA_MIN)
# rl_scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=2 * len(train_loader) // max(1, cfg.grad_accum_steps), T_mult=2, eta_min=ETA_MIN)

scaler = GradScaler(enabled=cfg_train.use_amp)

summary_writer = SummaryWriter(log_dir=cfg_train.log_dir, flush_secs=5)

# add model graph to tensorboard
# write model graph
dummy_input = torch.zeros((1, BLOCK_SIZE), dtype=torch.long, device=cfg_train.device)
summary_writer.add_graph(model.eval(), dummy_input)  # eval() avoids dropout noise
summary_writer.flush()
summary_writer.close()

# compile the model (can skip if not needed)
model.compile(mode="reduce-overhead")

# inspect the model
print(f"Model parameters: {count_params(model):_}")
print("vocab size: ", vocab_size)
print(f"excepted iniital CE loss( uniform model ): {-np.log(1/vocab_size):.4f}, ppl: {np.exp(-np.log(1/vocab_size)):.4f}")
# evaluate the untrained model
val_loss = evaluate(model, val_loader, device=cfg_train.device)
val_ppl = np.exp(val_loss)
print(f"not trained model Initial validation loss: {val_loss:.4f}, ppl: {val_ppl:.4f}")
print(model)

In [ ]:
train_results = train(model, train_loader, val_loader, cfg=cfg_train,
                      optimizer=optimizer, scheduler=lr_scheduler, scaler=scaler,
                      writer=summary_writer)

Training Progress:   0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1/60:   0%|          | 0/122 [00:00<?, ?it/s]

Epoch 2/60:   0%|          | 0/122 [00:00<?, ?it/s]

Epoch 3/60:   0%|          | 0/122 [00:00<?, ?it/s]

Epoch 4/60:   0%|          | 0/122 [00:00<?, ?it/s]

...

Epoch 15/60:   0%|          | 0/122 [00:00<?, ?it/s]

Epoch 16/60:   0%|          | 0/122 [00:00<?, ?it/s]

Epoch 17/60:   0%|          | 0/122 [00:00<?, ?it/s]

Epoch 18/60:   0%|          | 0/122 [00:00<?, ?it/s]

Epoch 19/60:   0%|          | 0/122 [00:00<?, ?it/s]

### save all parameters used in the model and training

In [ ]:
# save all the hyperparameters into a json file
rl_scheduler_state_dict = lr_scheduler.state_dict() if lr_scheduler.__class__.__name__ != "WarmupThenCosine" else {
            k:v for k, v in lr_scheduler.state_dict().items() if k != "cosine"}
all_hparams_cfg = {
    "tokenizer" : {
        "type": "bytelevel_bpe",
        "n_merges": N_MERGES,
        "min_frequency": MIN_FREQ,
        "special_tokens": SPECIAL_TOKENS,
    },
    "data": {
        "batch_size": BATCH_SIZE,
        "block_size": BLOCK_SIZE,
    },
    "model": vars(cfg_model).copy(),
    "optimizer": {
        "type": optimizer.__class__.__name__,
        "lr": LR,
        "weight_decay": WEIGHT_DECAY,
        "defaults": optimizer.defaults,
    },
    "lr_scheduler": {
        "type": lr_scheduler.__class__.__name__,
        "eta_min": ETA_MIN,
        "state_dict": rl_scheduler_state_dict,
    },
    "scaler": {
        "type": scaler.__class__.__name__,
        "enabled": scaler.is_enabled(),
        "state_dict": scaler.state_dict(),
    },
    "training": vars(cfg_train).copy(),
    "results": train_results,
}

# save it to checkpoint dir, same prefix as checkpoint files
with open(os.path.join(CHECKPOINT_DIR, all_hparams_cfg_filename), "w") as f:
    json.dump(all_hparams_cfg, f, indent=4)

# write it to tensorboard
summary_writer.add_text("config/json", "```json\n" + json.dumps(all_hparams_cfg, indent=2, sort_keys=True) + "\n```", global_step=0)
summary_writer.flush()
summary_writer.close()

## results and evaluation

In [ ]:
# make train_loss, train_ppl, val_ppl, val_loss and epochs into a DataFrame
history = train_results["history"]
history_df = pd.DataFrame(history)

display(history_df.head())
display(history_df.tail())
# plot results 
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_df['epochs'], history_df["train_loss"], label="Train Loss")
plt.plot(history_df['epochs'], history_df["val_loss"], label="Val Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(history_df['epochs'][::int(len(history_df['epochs'])//9)+1])  
plt.title("Training Loss and Validation Perplexity")
plt.legend()
plt.grid()

# plot perplexity
plt.subplot(1, 2, 2)
plt.plot(history_df['epochs'], history_df["train_ppl"], label="Train PPL")
plt.plot(history_df['epochs'], history_df["val_ppl"], label="Val PPL")
plt.xlabel("Epochs")
plt.ylabel("Perplexity")
plt.xticks(history_df['epochs'][::int(len(history_df['epochs'])//9)+1])  
plt.title("Validation Perplexity Over Time")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

NameError: name 'train_results' is not defined

### generate text from the trained model

In [ ]:
# compare with exponentiated average negative log likelihood
test_nll = evaluate(model, test_loader, cfg_train.device)
test_ppl = np.exp(test_nll)

val_ppl = train_results["best_val_ppl"]
train_ppl = train_results["best_train_ppl"]

val_nll = torch.log(torch.tensor(val_ppl)).item()
train_nll = torch.log(torch.tensor(train_ppl)).item()

# display in a DataFrame
test_results_df = pd.DataFrame({
    "split": ["train", "val", "test"],
    "loss(NLL)": [train_nll, val_nll, test_nll],
    "perplexity": [train_ppl, val_ppl, test_ppl],
})
display(test_results_df)

,split,loss(NLL),perplexity
0,train,3.629925,37.709980
1,val,3.404252,30.091774
2,test,3.465237,31.984036


In [ ]:
# get a batch from validation set and generate text
context_len = 10
batch_gen = 5
max_new_tokens = 100

context_ids_batch, _ = next(iter(val_loader))
context_ids_batch = context_ids_batch.to(cfg_train.device, non_blocking=True)
# generate text from the model
generated_text = model.generate(context_ids_batch[:batch_gen, :context_len], max_new_tokens=max_new_tokens, temperature=0.9, top_k=20)
print(f">>>> <context>\n", "-" *20, "\n<<<< <generated_text>")
for i, text in enumerate(generated_text):
    print("\n", "-" * 80)
    print(">>>>", decode(text[:context_len].tolist()))
    print("-" * 20)
    print("<<<< ", decode(text[context_len:].tolist()))
    if i + 1 >= batch_gen:
        break

>>>> <context>
 -------------------- 
<generated_text>

 --------------------------------------------------------------------------------
>>>> vil
Free me so far
--------------------
:

CLive




SCBRATIUS
BAllyems
TAVence,
TUS
He.
Hation,
HERe: apranian.

TZO
ORNTONAnders, and mortist is my say,



TORIt.
Fa, b

 --------------------------------------------------------------------------------
>>>> it on nature's misch
--------------------

Forteepratters offessiance,
Beman: for agian. The soul,
What should not dapres.
Whens of us,




DIUS






PAnsien
These and lad.

Caulage in thy make the per

 --------------------------------------------------------------------------------
>>>> s of success.



--------------------
There,
And,
TRDove to-s and tont, to dam; five.
Whathoubalk.
Younicep,--y, and gost be feet hathines.

How not.
DEODELEOfore mocksentss. Friark you, jeres

 --------------------------------------------------------------------------------
>>>>  Tuesday noon, or
--------